In [1]:
from google.colab import drive
drive.flush_and_unmount()


Drive not mounted, so nothing to flush and unmount.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/AOD-Net/baseline

/content/drive/MyDrive/AOD-Net/baseline


In [3]:
!ls

common.py      dehaze.py  orig_net.py  README.md  test.py     train.py
dataloader.py  net.py	  __pycache__  run.sh	  train_logs


In [ ]:
!python train.py  --help

usage: train.py [-h] [--dataset {ITS,OTS}] [--init_lr INIT_LR]
                [--milestones MILESTONES [MILESTONES ...]] [--gamma GAMMA]
                [--weight_decay WEIGHT_DECAY]
                [--grad_clip_norm GRAD_CLIP_NORM] [--loss_func LOSS_FUNC]
                [--num_epochs NUM_EPOCHS]
                [--train_batch_size TRAIN_BATCH_SIZE]
                [--val_batch_size VAL_BATCH_SIZE] [--num_workers NUM_WORKERS]
                [--display_iter DISPLAY_ITER] [--snapshot_iter SNAPSHOT_ITER]
                [--snapshots_folder SNAPSHOTS_FOLDER]
                [--sample_output_folder SAMPLE_OUTPUT_FOLDER]
                [--model_path MODEL_PATH] [--model_trained] [--valid_not_save]
                [--multi_gpu]

options:
  -h, --help            show this help message and exit
  --dataset {ITS,OTS}
  --weight_decay WEIGHT_DECAY
  --grad_clip_norm GRAD_CLIP_NORM
  --loss_func LOSS_FUNC
                        l1|l2
  --num_epochs NUM_EPOCHS
  --train_batch_size TRAIN_BATCH_

In [4]:
!python3 train.py --num_epochs 10 --train_batch_size 4 --val_batch_size 4

dataset :  ITS
init_lr :  0.0001
milestones :  [4, 7]
gamma :  0.1
weight_decay :  0.0001
grad_clip_norm :  0.1
loss_func :  l1
num_epochs :  10
train_batch_size :  4
val_batch_size :  4
num_workers :  4
display_iter :  10
snapshot_iter :  200
snapshots_folder :  /content/drive/MyDrive/AOD-Net/baseline/train_logs/snapshots/
sample_output_folder :  /content/drive/MyDrive/AOD-Net/baseline/train_logs/samples/
model_path :  /content/drive/MyDrive/AOD-Net/baseline/train_logs/snapshots/dehazer.pth
model_trained :  False
valid_not_save :  False
multi_gpu :  False
orig_images_path :  /content/drive/MyDrive/hazedataset/clear/
hazy_images_path :  /content/drive/MyDrive/hazedataset/hazy/

Start train!
Total training examples: 916
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Ple

In [5]:
!python3 test.py

model_path :  /content/drive/MyDrive/AOD-Net/baseline/train_logs/snapshots/dehazer.pth
test_output_folder :  ./test_results/
resize :  False
save_image :  False

Start Test!
iter 0:  1405.png  PSNR: 17.0864  SSIM 0.8631
iter 1:  1405.png  PSNR: 14.2893  SSIM 0.8006
iter 2:  1405.png  PSNR: 14.7613  SSIM 0.8128
iter 3:  1405.png  PSNR: 18.7173  SSIM 0.8907
iter 4:  1405.png  PSNR: 15.2674  SSIM 0.8251
iter 5:  1405.png  PSNR: 15.8150  SSIM 0.8375
iter 6:  1405.png  PSNR: 21.1593  SSIM 0.9234
iter 7:  1405.png  PSNR: 19.7764  SSIM 0.9060
iter 8:  1405.png  PSNR: 17.8418  SSIM 0.8766
iter 9:  1405.png  PSNR: 16.4158  SSIM 0.8500
iter 10:  1407.png  PSNR: 16.3022  SSIM 0.8697
iter 11:  1407.png  PSNR: 19.1392  SSIM 0.9040
iter 12:  1407.png  PSNR: 20.0549  SSIM 0.9123
iter 13:  1407.png  PSNR: 22.2561  SSIM 0.9278
iter 14:  1407.png  PSNR: 23.5728  SSIM 0.9347
iter 15:  1407.png  PSNR: 21.0910  SSIM 0.9203
iter 16:  1407.png  PSNR: 18.3150  SSIM 0.8954
iter 17:  1407.png  PSNR: 25.0600  SS

In [ ]:
%%writefile /content/drive/MyDrive/AOD-Net/baseline/test.py
import torch
import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import os
import sys
import argparse
import time
import numpy as np
from torchvision import transforms
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import cv2
import glob

import net
from common import config

def populate_test_list(gt_images_path, hazy_images_path):
    test_list = []
    image_list_haze = glob.glob(hazy_images_path + '*.jpg')
    image_list_haze.extend(glob.glob(hazy_images_path + '*.png'))
    tmp_dict ={}
    for image in image_list_haze:
        image = image.split('/')[-1]
        key = image.split('_')[0] + '.png'  # png maybe need be changed to jpg
        if key in tmp_dict.keys():
            tmp_dict[key].append(image)
        else:
            tmp_dict[key] = []
            tmp_dict[key].append(image)

    for key in tmp_dict.keys():
        for hazy_image in tmp_dict[key]:
            test_list.append([gt_images_path + key, hazy_images_path + hazy_image])

    return test_list


def eval_index(gt_img, dehaze_img):
    gt_img = gt_img.cpu().detach().numpy().transpose((0, 2, 3, 1))
    dehaze_img = dehaze_img.cpu().detach().numpy().transpose((0, 2, 3, 1))
    psnr = peak_signal_noise_ratio(gt_img[0], dehaze_img[0])
    ssim = structural_similarity(gt_img[0], dehaze_img[0], channel_axis=-1, data_range=1)
    return psnr, ssim


def test(test_list, args):
    dehaze_net = net.dehaze_net().cuda()
    dehaze_net.load_state_dict(torch.load(args.model_path))
    dehaze_net.eval()

    print('\nStart Test!')
    total_psnr = 0
    total_ssim = 0
    for ite, (img_gt, img_haze) in enumerate(test_list):
        img_name = img_gt
        img_gt = cv2.imread(img_gt)
        img_haze = cv2.imread(img_haze)
        img_gt = cv2.cvtColor(img_gt, cv2.COLOR_BGR2RGB)
        img_haze = cv2.cvtColor(img_haze, cv2.COLOR_BGR2RGB)
        if args.resize:
            img_gt = cv2.resize(img_gt, (config.height, config.width), cv2.INTER_LINEAR)
            img_haze = cv2.resize(img_haze, (config.height, config.width), cv2.INTER_LINEAR)
        img_gt = img_gt.transpose((2, 0, 1)) / 255.0
        img_haze = img_haze.transpose((2, 0, 1)) / 255.0
        img_gt = torch.from_numpy(img_gt).float().unsqueeze(0).cuda()
        img_haze = torch.from_numpy(img_haze).float().unsqueeze(0).cuda()
        with torch.no_grad():
            img_clean = dehaze_net(img_haze)
            img_clean = torch.clamp(img_clean, 0, 1)
        psnr, ssim = eval_index(img_gt, img_clean)
        total_psnr += psnr
        total_ssim += ssim
        print('iter %d: ' % ite, os.path.split(img_name)[-1], ' PSNR: %.4f  SSIM %.4f' % (psnr, ssim))

        if args.save_image:
            torchvision.utils.save_image(torch.cat((img_haze, img_clean, img_gt), 0),
                                     args.test_output_folder + os.path.split(img_name)[-1])

    num = len(test_list)
    total_psnr = total_psnr / num
    total_ssim = total_ssim / num
    print('Test numbers: %d' % num)
    print('Test Average PSNR: %.4f' % total_psnr)
    print('Test Average SSIM: %.4f' % total_ssim)

    return total_psnr, total_ssim


if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument('--model_path', type=str, default='/content/drive/MyDrive/AOD-Net/baseline/train_logs/snapshots/dehazer.pth')
    parser.add_argument('--test_output_folder', type=str, default="./test_results/")
    parser.add_argument('--resize', action='store_true')
    parser.add_argument('--save_image', action='store_true')

    args = parser.parse_args()
    for k, v in args.__dict__.items():
        print(k, ': ', v)

    if args.save_image and not os.path.exists(args.test_output_folder):
        os.system('mkdir -p %s' % args.test_output_folder)

    t1 = time.time()
    indoor_gt = '/content/drive/MyDrive/Reside_SOTS/indoor/clear/'
    indoor_hazy = '/content/drive/MyDrive/Reside_SOTS/indoor/hazy/'
    indoor_list = populate_test_list(indoor_gt, indoor_hazy)
    indoor_num = len(indoor_list)
    indoor_psnr, indoor_ssim = test(indoor_list, args)
    indoor_time = time.time() - t1

    t2 = time.time()
    outdoor_gt = '/content/drive/MyDrive/Reside_SOTS/outdoor/clear/'
    outdoor_hazy = '/content/drive/MyDrive/Reside_SOTS/outdoor/hazy/'
    outdoor_list = populate_test_list(outdoor_gt, outdoor_hazy)
    outdoor_num = len(outdoor_list)
    outdoor_psnr, outdoor_ssim = test(outdoor_list, args)
    outdoor_time = time.time() - t2


    print('\nIndoor time consume: %.2f' % indoor_time)
    print('Indoor test numbers: %d' % indoor_num)
    print('Indoor Average PSNR: %.4f' % indoor_psnr)
    print('Indoor Average SSIM: %.4f' % indoor_ssim)

    print('\nOutdoor time consume:%.2f' % outdoor_time)
    print('Outdoor test numbers: %d' % outdoor_num)
    print('Outdoor Average PSNR: %.4f' % outdoor_psnr)
    print('Outdoor Average SSIM: %.4f' % outdoor_ssim)

    total_psnr = (indoor_num*indoor_psnr + outdoor_num*outdoor_psnr) / (indoor_num + outdoor_num)
    total_ssim = (indoor_num*indoor_ssim + outdoor_num*outdoor_ssim) / (indoor_num + outdoor_num)
    print('\nTotal Test Average PSNR: %.4f' % total_psnr)
    print('Total Test Average SSIM: %.4f' % total_ssim)
    print('Test over!')





Writing /content/drive/MyDrive/AOD-Net/baseline/test.py


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AOD-Net/baseline/test.py'

In [ ]:
import os
os.makedirs('/content/drive/MyDrive/AOD-Net/baseline/', exist_ok=True)

In [ ]:
%%writefile /content/drive/MyDrive/AOD-Net/baseline/test.py
import torch
import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import os
import sys
import argparse
import time
import numpy as np
from torchvision import transforms
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import cv2
import glob

import net
from common import config

def populate_test_list(gt_images_path, hazy_images_path):
    test_list = []
    image_list_haze = glob.glob(hazy_images_path + '*.jpg')
    image_list_haze.extend(glob.glob(hazy_images_path + '*.png'))
    tmp_dict ={}
    for image in image_list_haze:
        image = image.split('/')[-1]
        key = image.split('_')[0] + '.png'  # png maybe need be changed to jpg
        if key in tmp_dict.keys():
            tmp_dict[key].append(image)
        else:
            tmp_dict[key] = []
            tmp_dict[key].append(image)

    for key in tmp_dict.keys():
        for hazy_image in tmp_dict[key]:
            test_list.append([gt_images_path + key, hazy_images_path + hazy_image])

    return test_list


def eval_index(gt_img, dehaze_img):
    gt_img = gt_img.cpu().detach().numpy().transpose((0, 2, 3, 1))
    dehaze_img = dehaze_img.cpu().detach().numpy().transpose((0, 2, 3, 1))
    psnr = peak_signal_noise_ratio(gt_img[0], dehaze_img[0])
    ssim = structural_similarity(gt_img[0], dehaze_img[0], channel_axis=-1, data_range=1)
    return psnr, ssim


def test(test_list, args):
    dehaze_net = net.dehaze_net().cuda()
    dehaze_net.load_state_dict(torch.load(args.model_path))
    dehaze_net.eval()

    print('\nStart Test!')
    total_psnr = 0
    total_ssim = 0
    for ite, (img_gt, img_haze) in enumerate(test_list):
        img_name = img_gt
        img_gt = cv2.imread(img_gt)
        img_haze = cv2.imread(img_haze)
        img_gt = cv2.cvtColor(img_gt, cv2.COLOR_BGR2RGB)
        img_haze = cv2.cvtColor(img_haze, cv2.COLOR_BGR2RGB)
        if args.resize:
            img_gt = cv2.resize(img_gt, (config.height, config.width), cv2.INTER_LINEAR)
            img_haze = cv2.resize(img_haze, (config.height, config.width), cv2.INTER_LINEAR)
        img_gt = img_gt.transpose((2, 0, 1)) / 255.0
        img_haze = img_haze.transpose((2, 0, 1)) / 255.0
        img_gt = torch.from_numpy(img_gt).float().unsqueeze(0).cuda()
        img_haze = torch.from_numpy(img_haze).float().unsqueeze(0).cuda()
        with torch.no_grad():
            img_clean = dehaze_net(img_haze)
            img_clean = torch.clamp(img_clean, 0, 1)
        psnr, ssim = eval_index(img_gt, img_clean)
        total_psnr += psnr
        total_ssim += ssim
        print('iter %d: ' % ite, os.path.split(img_name)[-1], ' PSNR: %.4f  SSIM %.4f' % (psnr, ssim))

        if args.save_image:
            torchvision.utils.save_image(torch.cat((img_haze, img_clean, img_gt), 0),
                                     args.test_output_folder + os.path.split(img_name)[-1])

    num = len(test_list)
    total_psnr = total_psnr / num
    total_ssim = total_ssim / num
    print('Test numbers: %d' % num)
    print('Test Average PSNR: %.4f' % total_psnr)
    print('Test Average SSIM: %.4f' % total_ssim)

    return total_psnr, total_ssim


if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument('--model_path', type=str, default='/content/drive/MyDrive/AOD-Net/baseline/train_logs/snapshots/dehazer.pth')
    parser.add_argument('--test_output_folder', type=str, default="./test_results/")
    parser.add_argument('--resize', action='store_true')
    parser.add_argument('--save_image', action='store_true')

    args = parser.parse_args()
    for k, v in args.__dict__.items():
        print(k, ': ', v)

    if args.save_image and not os.path.exists(args.test_output_folder):
        os.system('mkdir -p %s' % args.test_output_folder)

    t1 = time.time()
    indoor_gt = '/content/drive/MyDrive/Reside_SOTS/indoor/clear/'
    indoor_hazy = '/content/drive/MyDrive/Reside_SOTS/indoor/hazy/'
    indoor_list = populate_test_list(indoor_gt, indoor_hazy)
    indoor_num = len(indoor_list)
    indoor_psnr, indoor_ssim = test(indoor_list, args)
    indoor_time = time.time() - t1

    t2 = time.time()
    outdoor_gt = '/content/drive/MyDrive/Reside_SOTS/outdoor/clear/'
    outdoor_hazy = '/content/drive/MyDrive/Reside_SOTS/outdoor/hazy/'
    outdoor_list = populate_test_list(outdoor_gt, outdoor_hazy)
    outdoor_num = len(outdoor_list)
    outdoor_psnr, outdoor_ssim = test(outdoor_list, args)
    outdoor_time = time.time() - t2


    print('\nIndoor time consume: %.2f' % indoor_time)
    print('Indoor test numbers: %d' % indoor_num)
    print('Indoor Average PSNR: %.4f' % indoor_psnr)
    print('Indoor Average SSIM: %.4f' % indoor_ssim)

    print('\nOutdoor time consume:%.2f' % outdoor_time)
    print('Outdoor test numbers: %d' % outdoor_num)
    print('Outdoor Average PSNR: %.4f' % outdoor_psnr)
    print('Outdoor Average SSIM: %.4f' % outdoor_ssim)

    total_psnr = (indoor_num*indoor_psnr + outdoor_num*outdoor_psnr) / (indoor_num + outdoor_num)
    total_ssim = (indoor_num*indoor_ssim + outdoor_num*outdoor_ssim) / (indoor_num + outdoor_num)
    print('\nTotal Test Average PSNR: %.4f' % total_psnr)
    print('Total Test Average SSIM: %.4f' % total_ssim)
    print('Test over!')